In [ ]:
!pip install -q faiss-cpu langchain langchain-core langchain-community
!pip install -q sentence-transformers
!pip install -q llama-cpp-python huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00


In [ ]:
# RAG推薦系統 - Google Colab版本
import os, re
from huggingface_hub import list_repo_files, hf_hub_download
from langchain_community.llms import LlamaCpp
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

#掛載Google Drive
from google.colab import drive
drive.mount('/content/drive')

#設定路徑
FAISS_DIR = "/content/drive/MyDrive/data/rag_faiss_demo1/faiss_index" #要確認
GGUF_REPO = "floraliuya/recft_unsloth-Meta-Llama-3.1-8B-2"
MODEL_FILE = "unsloth.Q4_K_M.gguf"  # 請根據實際的 GGUF 檔名調整


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#下載 GGUF 模型
def download_gguf_model(repo_id, filename, local_dir="./models/"):
    """下載 GGUF 模型檔案"""
    print(f"正在下載模型: {repo_id}/{filename}")

    if not os.path.exists(local_dir):
        os.makedirs(local_dir)

    try:
        model_path = hf_hub_download(
            repo_id=repo_id,
            filename=filename,
            local_dir=local_dir,
            local_dir_use_symlinks=False
        )
        print(f"模型下載完成: {model_path}")
        return model_path
    except Exception as e:
        print(f"下載失敗: {e}")
        return None

# 下載模型
model_path = download_gguf_model(GGUF_REPO, MODEL_FILE)

#初始化 Llama CPP 模型
def initialize_llama_model(model_path):
    """初始化 LlamaCpp 模型"""
    print("初始化 Llama CPP 模型...")

    try:
        llm = LlamaCpp(
            model_path=model_path,
            temperature=0.7,
            max_tokens=512,
            top_p=0.9,
            n_ctx=4096,  # context 長度
            n_gpu_layers=35,  # 使用 GPU 加速的層數，根據你的 GPU 記憶體調整
            verbose=True,
            seed=42,
            n_threads=4
        )
        print("模型載入成功！")
        return llm
    except Exception as e:
        print(f"模型載入失敗: {e}")
        return None

# 初始化模型
if model_path:
    llm = initialize_llama_model(model_path)
else:
    print("模型下載失敗，無法繼續")
    llm = None


正在下載模型: floraliuya/recft_unsloth-Meta-Llama-3.1-8B-2/unsloth.Q4_K_M.gguf


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

模型下載完成: models/unsloth.Q4_K_M.gguf
初始化 Llama CPP 模型...


llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from models/unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Recft_Unsloth Meta Llama 3.1 8B 2
llama_model_loader: - kv   3:                            general.version str              = 2
llama_model_loader: - kv   4:                           general.basename str              = recft_unsloth-Meta-Llama-3.1
llama_model_loader: - kv   5:                       general.quantized_by str              = Unsloth
llama_model_loader: - kv   6:                         general.size_label str              = 8B
llama_model_loader: - kv   7:     

模型載入成功！


CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | BMI2 = 1 | AVX512 = 1 | LLAMAFILE = 1 | OPENMP = 1 | REPACK = 1 | 
Model metadata: {'general.file_type': '15', 'tokenizer.ggml.eos_token_id': '128001', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'gpt2', 'llama.vocab_size': '128256', 'llama.attention.value_length': '128', 'llama.attention.key_length': '128', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.freq_base': '500000.000000', 'general.architecture': 'llama', 'tokenizer.ggml.add_sep_token': 'false', 'llama.attention.head_count_kv': '8', 'llama.block_count': '32', 'tokenizer.ggml.padding_token_id': '128004', 'general.basename': 'recft_unsloth-Meta-Llama-3.1', 'tokenizer.ggml.bos_token_id': '128000', 'llama.attention.head_count': '32', 'tokenizer.ggml.pre': 'llama-bpe', 'llama.context_length': '131072', 'general.name': 'Recft_Unsloth Meta Llama 3.1 8B 2', 'llama.rope.dimension_count': '128', 'general.version': '2', 'general.re

In [ ]:

#載入向量資料庫
def load_faiss_database():
    """載入 FAISS 向量資料庫"""
    print("載入FAISS向量資料庫...")
    try:
        # 載入embedding模型 (需要與建立FAISS時使用相同的模型)
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-mpnet-base-v2", #請根據您實際使用的embedding模型調整
            #"sentence-transformers/all-MiniLM-L6-v2"
            model_kwargs={'device': 'cuda' if os.environ.get('COLAB_GPU') else 'cpu'}
        )

        # 載入FAISS向量資料庫
        faiss_db = FAISS.load_local(
            FAISS_DIR,
            embeddings,
            allow_dangerous_deserialization=True
        )
        print(f"FAISS 資料庫載入成功，包含 {faiss_db.index.ntotal} 個向量")
        return faiss_db
    except Exception as e:
        print(f"FAISS 資料庫載入失敗: {e}")
        return None

faiss_db = load_faiss_database()

載入FAISS向量資料庫...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS 資料庫載入成功，包含 105542 個向量


In [ ]:
#設定Alpaca prompt模板
alpaca_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Based on the provided context, give product recommendations that match the user's query. Focus on relevant features and explain why these products are suitable. Keep your response concise and helpful.

### Input:
User Query: {question}

Context from database: {context}

### Response:
"""

prompt = PromptTemplate.from_template(alpaca_prompt_template)

In [ ]:
#RAG推薦系統主函數
def get_recommendations(question, k=3, max_context_length=2000):
    """
    RAG推薦系統主函數

    Args:
        question (str): 用戶查詢
        k (int): 檢索的相似文檔數量
        max_context_length (int): 最大 context 長度

    Returns:
        str: 推薦結果
    """
    if not llm or not faiss_db:
        print("模型或資料庫未正確載入")
        return None

    print(f"處理查詢: {question}")

    try:
        # 搜尋相似文檔
        print("搜尋相似產品...")
        found_docs = faiss_db.similarity_search(question, k=k)

        # 準備context，限制長度避免超出模型限制
        context_parts = []
        current_length = 0

        for i, doc in enumerate(found_docs):
            doc_text = f"Product {i+1}: {doc.page_content}"
            if current_length + len(doc_text) < max_context_length:
                context_parts.append(doc_text)
                current_length += len(doc_text)
            else:
                break

        context = "\n\n".join(context_parts)

        # 構建完整prompt
        full_prompt = prompt.format(question=question, context=context)

        print("生成推薦...")
        # 使用 LlamaCpp 生成回應
        response = llm.invoke(full_prompt)

        return response

    except Exception as e:
        print(f"推理過程發生錯誤: {e}")
        return None

In [ ]:
#一邊生成、一邊即時把字吐出來
def get_recommendations_stream(question, k=3):
    """串流生成推薦結果"""
    if not llm or not faiss_db:
        print("模型或資料庫未正確載入")
        return

    print(f"處理查詢: {question}")

    try:
        found_docs = faiss_db.similarity_search(question, k=k)
        context = "\n\n".join([f"Product {i+1}: {doc.page_content}" for i, doc in enumerate(found_docs)])
        full_prompt = prompt.format(question=question, context=context)

        print("生成推薦 (串流模式):")
        print("-" * 50)

        # 使用串流生成
        for chunk in llm.stream(full_prompt):
            print(chunk, end="", flush=True)

        print("\n" + "-" * 50)

    except Exception as e:
        print(f"串流生成失敗: {e}")


In [ ]:
#測試範例
def run_test_queries():
    """執行測試查詢"""
    test_queries = [
        "I'm looking for a yellow summer dress which is light and airy",
        "Recommend comfortable shoes suitable for sports.",
        "Find some warm coats suitable for winter."
    ]

    for query in test_queries:
        print("=" * 60)
        print(f"查詢: {query}")
        print("=" * 60)

        result = get_recommendations(query, k=3)
        if result:
            print("推薦結果:")
            print(result)
        else:
            print("推薦生成失敗")

        print("=" * 60)
        print()

In [ ]:
#輔助函數
def check_model_info():
    """檢查模型資訊"""
    if llm:
        print("模型資訊:")
        print(f"模型路徑: {llm.model_path}")
        print(f"上下文長度: {llm.n_ctx}")
        print(f"最大生成長度: {llm.max_tokens}")
        print(f"使用GPU層數: {llm.n_gpu_layers}")

def explore_database(sample_size=5):
    """探索資料庫內容"""
    if not faiss_db:
        print("資料庫未載入")
        return

    print("資料庫探索:")
    print(f"總文檔數: {faiss_db.index.ntotal}")

    # 隨機搜尋一些文檔來查看內容結構
    sample_query = "product"
    docs = faiss_db.similarity_search(sample_query, k=sample_size)

    for i, doc in enumerate(docs):
        print(f"\n--- 文檔 {i+1} ---")
        content = doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content
        print(content)


In [ ]:
#主執行區塊
if __name__ == "__main__":
    print("RAG推薦系統 - Llama CPP版本")
    print("=" * 60)

    # 檢查系統狀態
    if llm and faiss_db:
        print("✓ 系統初始化完成")
        check_model_info()
        explore_database()

        # 執行測試
        print("\n開始測試...")
        run_test_queries()

    else:
        print("✗ 系統初始化失敗，請檢查模型和資料庫路徑")


RAG推薦系統 - Llama CPP版本
✓ 系統初始化完成
模型資訊:
模型路徑: models/unsloth.Q4_K_M.gguf
上下文長度: 4096
最大生成長度: 512
使用GPU層數: 35
資料庫探索:
總文檔數: 105542

--- 文檔 1 ---
JULIUS HT BB - Sneakers - Black - Children Accessories, Swimwear - nan

--- 文檔 2 ---
PRICED ITEM tee - T-shirt - Black - Sport - Short-sleeved sports top in printed, fast-drying functional fabric.

--- 文檔 3 ---
Borg WL sneaker - Sneakers - Black - Ladies Accessories - nan

開始測試...
查詢: I'm looking for a yellow summer dress which is light and airy
處理查詢: I'm looking for a yellow summer dress which is light and airy
搜尋相似產品...
生成推薦...


llama_perf_context_print:        load time =   70521.23 ms
llama_perf_context_print: prompt eval time =   70514.96 ms /   230 tokens (  306.59 ms per token,     3.26 tokens per second)
llama_perf_context_print:        eval time =  213992.91 ms /   367 runs   (  583.09 ms per token,     1.72 tokens per second)
llama_perf_context_print:       total time =  285903.13 ms /   597 tokens
llama_perf_context_print:    graphs reused =        376
Llama.generate: 72 prefix-match hit, remaining 109 prompt tokens to eval


推薦結果:
Based on the provided context, I recommend the following products:

1. Product Name: Covent Garden - Dress - Light Yellow - Ladieswear
   Description: A sleeveless maxi dress in crinkled viscose with inset lace sections, crocheted lace shoulder straps and a seam with flounces at the hem.
   Features: Viscose fabric with crinkled texture for a lightweight and airy feel. Inset lace panels provide a delicate touch of elegance. Crocheted lace shoulder straps add a romantic flair to the dress. The seam with flounces at the hem adds an element of femininity and gracefulness.

2. Product Name: SUMMER STRAP DRESS_09-090
   Description: A short dress in soft jersey with a V-neck, narrow, adjustable shoulder straps, seam at the waist and flared skirt.
   Features: Jersey fabric that is lightweight and comfortable to wear during the summer months. The V-neckline adds a touch of femininity to the dress. Adjustable shoulder straps ensure a perfect fit for various body types. The seam at the w

llama_perf_context_print:        load time =   70521.23 ms
llama_perf_context_print: prompt eval time =   30776.39 ms /   109 tokens (  282.35 ms per token,     3.54 tokens per second)
llama_perf_context_print:        eval time =  164993.18 ms /   286 runs   (  576.90 ms per token,     1.73 tokens per second)
llama_perf_context_print:       total time =  196824.13 ms /   395 tokens
llama_perf_context_print:    graphs reused =        285
Llama.generate: 72 prefix-match hit, remaining 252 prompt tokens to eval


推薦結果:
Outfit 1:
- Shoes: Harris cross sandal PQ - Sandals - Black
- Bottoms: Lightweight cotton trousers in a light color such as white or beige
- Top: A breathable and lightweight polo shirt in a solid color, preferably in the same tone as your bottom.
- Accessories: Add a pair of sunglasses with a sporty design, as well as a small crossbody bag for carrying essentials.

Outfit 2:
- Shoes: Steel Court sneaker - Sneakers - Beige
- Bottoms: Lightweight and breathable running shorts or athletic leggings in a neutral color like black, navy blue, or grey.
- Top: Choose a moisture-wicking sports tank top or crew neck t-shirt in a dark color to provide sun protection for your upper body.
- Accessories: Add a pair of performance sunglasses with UV protection, as well as a small fanny pack for carrying essentials during outdoor activities.

Outfit 3:
- Shoes: Marc Pq sandal - Sandals - Black
- Bottoms: Lightweight and breathable linen shorts or cargo pants in a neutral color like khaki or tan.

llama_perf_context_print:        load time =   70521.23 ms
llama_perf_context_print: prompt eval time =   74697.76 ms /   252 tokens (  296.42 ms per token,     3.37 tokens per second)
llama_perf_context_print:        eval time =  298736.68 ms /   511 runs   (  584.61 ms per token,     1.71 tokens per second)
llama_perf_context_print:       total time =  375276.30 ms /   763 tokens
llama_perf_context_print:    graphs reused =        517


推薦結果:
Outfit 1: 
- Top: A soft, long-sleeved sweater in a cozy neutral shade such as cream or beige. The fabric should be lightweight yet warm enough for winter.
- Bottom: Pair the sweater with a pair of straight-leg jeans in a dark wash to create a modern and chic look.
- Shoes: Complete the outfit with a pair of ankle boots in a sleek black leather. These will provide both warmth and style during the cold winter months.

Outfit 2:
- Top: Opt for a fitted, long-sleeved turtleneck sweater in a bold color such as burgundy or emerald green. This will add an element of sophistication to your outfit.
- Bottom: Pair the turtleneck sweater with a pair of high-waisted leggings in a neutral shade like charcoal gray or black. The high waistline will elongate your silhouette and make you appear taller.
- Shoes: Finish off the outfit with a pair of ankle boots in a metallic finish such as silver or gold. This will add a touch of glamour to your overall look.
- Accessories: To complete the ensembl

In [ ]:
#互動式查詢
def interactive_query():
    """互動式查詢介面"""
    if not llm or not faiss_db:
        print("系統未正確初始化")
        return

    print("進入互動模式 (輸入 'quit' 退出)")
    while True:
        user_input = input("\n請輸入您的查詢: ")
        if user_input.lower() == 'quit':
            break

        if user_input.strip():
            result = get_recommendations(user_input, k=3)
            if result:
                print(f"\n推薦結果:\n{result}")

# 取消註解以使用互動模式
interactive_query()

進入互動模式 (輸入 'quit' 退出)

請輸入您的查詢: I’m looking for a formal shirt made of linen
處理查詢: I’m looking for a formal shirt made of linen
搜尋相似產品...
生成推薦...


Llama.generate: 72 prefix-match hit, remaining 233 prompt tokens to eval
llama_perf_context_print:        load time =   70521.23 ms
llama_perf_context_print: prompt eval time =   64207.30 ms /   233 tokens (  275.57 ms per token,     3.63 tokens per second)
llama_perf_context_print:        eval time =  273003.45 ms /   470 runs   (  580.86 ms per token,     1.72 tokens per second)
llama_perf_context_print:       total time =  338860.17 ms /   703 tokens
llama_perf_context_print:    graphs reused =        476



推薦結果:
Hello there! I'm here to provide some recommendations for the perfect formal shirt made of linen.

Product 1: Premium My Linen Shirt
This shirt is perfect for a formal occasion. The premium quality linen fabric ensures breathability and comfort, making it ideal for warmer months or outdoor events.

The V-neckline adds a touch of sophistication while still maintaining a relaxed and effortless style.

With its classic design elements like the concealed button-down collar, open chest pocket, long sleeves with adjustable buttoning at the cuffs, and a yoke with pleat at the back. This shirt exudes timeless elegance.

The slim fit option ensures that it flatters your figure and gives you a polished look for any formal occasion.

Product 2: lucas linen price
This shirt is perfect for a formal event or a dressy night out. The beige color adds a touch of sophistication while the premium quality linen fabric ensures breathability and comfort, making it ideal for warmer months or outdoor e

Llama.generate: 74 prefix-match hit, remaining 147 prompt tokens to eval
llama_perf_context_print:        load time =   70521.23 ms
llama_perf_context_print: prompt eval time =   41503.25 ms /   147 tokens (  282.34 ms per token,     3.54 tokens per second)
llama_perf_context_print:        eval time =  106517.53 ms /   187 runs   (  569.61 ms per token,     1.76 tokens per second)
llama_perf_context_print:       total time =  148667.84 ms /   334 tokens
llama_perf_context_print:    graphs reused =        193



推薦結果:
Hello User,

Based on your requirements and context from the database, I recommend the following products:

1. Simone tank dress (Product 1): This fitted, sleeveless dress in soft jersey is perfect for a formal yet casual event like a graduation party. The gathers around the sides ensure a comfortable fit.

2. Simone tank dress (Product 2): Another great option from the same brand is this dark turquoise colored tank dress. It has all the features of Product 1 but with an edgier color choice.

3. Neives Short Fluffy M-Gown (J) (Product 3): This dressing gown in soft, fluffy velour is perfect for a formal yet casual lunch vibe. The hood and detachable tie belt at the waist provide added functionality and versatility.

I hope these recommendations help you find the right dress for your friend’s graduation party. Please let me know if you have any further questions or concerns.

請輸入您的查詢: quit
